In [1]:
#!+linux操作命令
#!pip list
#!pip install --upgrade pip
#!pip install --upgrade numpy==1.15.2

# Tensor, Computational graph, Session

In [2]:
#张量，计算图，会话
import tensorflow as tf
#三种方式定义变量

a = tf.constant([[1.0, 2.0],[2.0, 3.0]], name="a")
print(a)
b = tf.constant([[1.0, 2.0],[2.0, 3.0]], name="b")

result = a + b
print(result)

sess = tf.Session()
print(sess.run(result))
sess.close()

#用这个
with tf.Session() as sess1:
    print(sess1.run(result))

Tensor("a:0", shape=(2, 2), dtype=float32)
Tensor("add:0", shape=(2, 2), dtype=float32)
[[2. 4.]
 [4. 6.]]
[[2. 4.]
 [4. 6.]]


In [3]:
#coding:utf-8
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    #在计算图g1中定义一个变量v,并且设置初始值为0
    v1 = tf.get_variable(name="v1", initializer=tf.zeros_initializer( )(shape=[2,2]))
    v2 = tf.get_variable(name="v2", initializer=tf.zeros_initializer( )(shape=[2,2]))
    a = tf.constant([1])

g2 = tf.Graph()
with g2.as_default():
    # 在计算图g2中定义一个变量v,并且设置初始值为0
    v = tf.get_variable(name="v", initializer=tf.ones_initializer( )(shape=[1]))
    b = tf.constant([0])

with tf.Session(graph=g1) as sess:
    # 运行初始化的两种方法
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(a)
    #这句话是什么意思？
    with tf.variable_scope("",reuse=True):
        #通过名字获取变量
        print(sess.run(a))

with tf.Session(graph=g2) as sess:
    # 运行初始化的两种方法
    init = tf.global_variables_initializer()
    sess.run(init)
    #tf.initialize_all_variables().run()

    #这句话是什么意思？
    with tf.variable_scope("",reuse=True):
        #通过名字获取变量
        print(sess.run(tf.get_variable("v")))
        print(sess.run(b))

[1]
[1.]
[0]


# A simple NN application by TensorFlow

In [4]:
import numpy as np
BATCH_SIZE = 8
SEED = 23450

rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
Y_ = [[x1 + x2 + (rdm.rand()/10.0 - 0.05)] for (x1, x2) in X]

x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w)

loss_mse = tf.reduce_mean(tf.square(y_ - y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss_mse)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 20000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = (i*BATCH_SIZE) % 32 + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y_[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w is: " % (i))
            print(sess.run(w))
    print("Final w is: \n", sess.run(w))

After 0 training steps, w is: 
[[-0.7992299]
 [ 1.493489 ]]
After 500 training steps, w is: 
[[0.5660787]
 [1.3736113]]
After 1000 training steps, w is: 
[[0.85490614]
 [1.1255193 ]]
After 1500 training steps, w is: 
[[0.9535033]
 [1.0405437]]
After 2000 training steps, w is: 
[[0.9872092]
 [1.0114942]]
After 2500 training steps, w is: 
[[0.99873126]
 [1.001564  ]]
After 3000 training steps, w is: 
[[1.0026709]
 [0.9981688]]
After 3500 training steps, w is: 
[[1.0040169]
 [0.9970085]]
After 4000 training steps, w is: 
[[1.0044775]
 [0.9966117]]
After 4500 training steps, w is: 
[[1.0046326]
 [0.9964766]]
After 5000 training steps, w is: 
[[1.0046859]
 [0.996431 ]]
After 5500 training steps, w is: 
[[1.0047067]
 [0.996414 ]]
After 6000 training steps, w is: 
[[1.0047106]
 [0.9964104]]
After 6500 training steps, w is: 
[[1.0047106]
 [0.9964104]]
After 7000 training steps, w is: 
[[1.0047106]
 [0.9964104]]
After 7500 training steps, w is: 
[[1.0047106]
 [0.9964104]]
After 8000 training st

# Handwritten digit recognition

In [5]:
#神经网络——手写数字识别
import tensorflow as tf
# load mnist data

#使用这一行来下载代码
#one_hot就是数据的标签形式，例如5就是5,[0,0,0,0,0,1,0,0,0,0,0]
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

#下载并加载数据
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

#数据与标签的占位
x = tf.placeholder(tf.float32,shape = [None,784])
y_actual = tf.placeholder(tf.float32,shape=[None,10])

#初始化权重和偏置
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

#softmax回归，得到预测概率
y_predict = tf.nn.softmax(tf.matmul(x,W) + b)
#y = 1/n[cross_entropy1,cross_entropy2,cross_entropy3][]
#求交叉熵得到残差，reduction_indices=1以1这个维度来塌缩求交叉熵，也就是矩阵横轴这个维度
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_actual*tf.log(y_predict),reduction_indices=1))
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_predict, labels=tf.argmax(y_actual, 1))

#梯度下降法使得残差最小
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

#测试阶段，测试准确度计算，1也是表示1这个维度，就是矩阵横轴这个维度
correct_prediction = tf.equal(tf.argmax(y_predict,1),tf.argmax(y_actual,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))#多个批次的准确度均值

#初始化值的变量还没有真正运行，只是搭建了一个计算图，使用这个方法就是运行了
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    #训练，迭代1000次
    for i in range(1000):
        batch_xs,batch_ys = mnist.train.next_batch(100)#按批次训练，每批100行数据
        sess.run(train_step,feed_dict={x:batch_xs,y_actual:batch_ys})#执行训练
        if(i%100==0):#每训练100次，测试一次
            print("accuracy:",sess.run(accuracy,feed_dict={x: mnist.test.images, y_actual: mnist.test.labels}))

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
accuracy: 0.3385
accuracy: 0.8162
accuracy: 0.873
accuracy: 0.896
accuracy: 0.9035
accuracy: 0.9067
accuracy: 0.9101
accuracy: 0.9104
accuracy: 0.9119
accuracy: 0.9129


In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_NODE = 784  # 输入节点
OUTPUT_NODE = 10  # 输出节点
LAYER1_NODE = 100  # 隐藏层数

BATCH_SIZE = 100  # 每次batch打包的样本个数
TRAINING_STEPS = 5000


def train(mnist):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([784, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, 10], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[10]))

    # 计算不含滑动平均类的前向传播结果
    layer1 = tf.nn.relu(tf.matmul(x, weights1) + biases1)
    y = tf.matmul(layer1, weights2) + biases2

    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    #cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy_mean)

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 初始化会话，并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}

        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))

            xs, ys = mnist.train.next_batch(50)
            sess.run(train_step, feed_dict={x: xs, y_: ys})

        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" % (TRAINING_STEPS, test_acc)))


def main():
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    train(mnist)


if __name__ == '__main__':
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1092 
After 1000 training step(s), validation accuracy using average model is 0.8668 
After 2000 training step(s), validation accuracy using average model is 0.892 
After 3000 training step(s), validation accuracy using average model is 0.9072 
After 4000 training step(s), validation accuracy using average model is 0.9148 
After 5000 training step(s), test accuracy using average model is 0.9184
